In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from google.colab import drive
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Sentiment Analysis/emoevent_en.csv'
df = pd.read_csv(file_path, delimiter='\t')
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(df['tweet'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors='tf')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenized['input_ids'].numpy()
y = df['emotion'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
183/183 [==============================] - 203s 848ms/step - loss: 1.3735 - accuracy: 0.4923
Epoch 2/10
183/183 [==============================] - 158s 865ms/step - loss: 0.9982 - accuracy: 0.6347
Epoch 3/10
183/183 [==============================] - 158s 865ms/step - loss: 0.6727 - accuracy: 0.7588
Epoch 4/10
183/183 [==============================] - 158s 862ms/step - loss: 0.4004 - accuracy: 0.8675
Epoch 5/10
183/183 [==============================] - 158s 863ms/step - loss: 0.2345 - accuracy: 0.9259
Epoch 6/10
183/183 [==============================] - 158s 866ms/step - loss: 0.1361 - accuracy: 0.9591
Epoch 7/10
183/183 [==============================] - 159s 867ms/step - loss: 0.0978 - accuracy: 0.9690
Epoch 8/10
183/183 [==============================] - 158s 865ms/step - loss: 0.0766 - accuracy: 0.9764
Epoch 9/10
183/183 [==============================] - 158s 865ms/step - loss: 0.0633 - accuracy: 0.9813
Epoch 10/10
183/183 [==============================] - 158s 865m

In [ ]:
predictions = model.predict(X_test).logits
predicted_labels = [np.argmax(p) for p in predictions]
print(classification_report(y_test, predicted_labels))

46/46 [==============================] - 16s 284ms/step
              precision    recall  f1-score   support

           0       0.21      0.15      0.18        71
           1       0.38      0.44      0.41       152
           2       0.25      0.21      0.23        28
           3       0.69      0.58      0.63       412
           4       0.59      0.70      0.64       653
           5       0.67      0.36      0.47        97
           6       0.29      0.19      0.23        48

    accuracy                           0.57      1461
   macro avg       0.44      0.38      0.40      1461
weighted avg       0.57      0.57      0.56      1461



In [ ]:
print(label_encoder.classes_)

['anger' 'disgust' 'fear' 'joy' 'others' 'sadness' 'surprise']
